In [4]:
#pip install pyathena

In [5]:
#pip install holidays

In [6]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt

In [7]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name = 'analytics'
table_name = 'daily_check'

In [8]:
#Connection

conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

In [9]:
df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

In [10]:
df = df[df['amount'] != 0] # Elimino 0

In [11]:
df['date'] = pd.to_datetime(df['date'])

In [14]:
# Extract year from the 'date' column
df['year'] = df['date'].dt.year

In [15]:
df['year'].unique()

array([2023, 2021, 2022, 2020])

In [16]:
df = df[df['year'] != 2020]

In [17]:
df['payer_country'] = df['payer']+'_'+df['country'] # Agrupo

/tmp/ipykernel_20/3820957550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['payer_country'] = df['payer']+'_'+df['country'] # Agrupo


In [18]:
df.head()

,payer,country,date,tx,amount,year,payer_country
0,TRANSFERTO - THUNES,BELGIUM,2023-04-03,1,55.55,2023,TRANSFERTO - THUNES_BELGIUM
2,BANCO GANADERO (BOLIVIA),BOLIVIA,2023-03-27,28,11802.00,2023,BANCO GANADERO (BOLIVIA)_BOLIVIA
3,BANCO DEL AUSTRO,ECUADOR,2021-09-10,213,78791.19,2021,BANCO DEL AUSTRO_ECUADOR
4,FEDECACES (EL SALVADOR),EL SALVADOR,2022-04-20,223,148547.00,2022,FEDECACES (EL SALVADOR)_EL SALVADOR
5,CARIBE EXPRESS,DOMINICAN REPUBLIC,2023-01-23,1117,350050.40,2023,CARIBE EXPRESS_DOMINICAN REPUBLIC


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92938 entries, 0 to 101969
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   payer          92938 non-null  object        
 1   country        92938 non-null  object        
 2   date           92938 non-null  datetime64[ns]
 3   tx             92938 non-null  int64         
 4   amount         92938 non-null  float64       
 5   year           92938 non-null  int64         
 6   payer_country  92938 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 5.7+ MB


In [20]:
df.nunique()

payer              133
country             75
date              1024
tx                5685
amount           74855
year                 3
payer_country      231
dtype: int64

In [21]:
#countries_list = df['country'].unique()

In [22]:
# Weekday indicator (dummy)
df['weekday'] = (df['date'].dt.dayofweek < 4).astype(int)

/tmp/ipykernel_20/1510508162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weekday'] = (df['date'].dt.dayofweek < 4).astype(int)


In [ ]:
holidays_by_country = { # Mothers day, thanksgiving, christmas, new year, US independence day
    'MEXICO': ['05-10', 'Thanksgiving', '12-25', '01-01', '07-04'],  
    'GUATEMALA': ['05-10', 'Thanksgiving' '12-25', '01-01', '07-04'],  
    'HONDURAS': ['05-10', 'Thanksgiving' '12-25', '01-01', '07-04'],  
    'EL SALVADOR': ['05-10', 'Thanksgiving' '12-25', '01-01', '07-04'],  
    'BRAZIL': ['Second Sunday in May', 'Thanksgiving', '12-25', '01-01', '07-04'],  
    'COLOMBIA': ['Second Sunday in May', 'Thanksgiving', '12-25', '01-01', '07-04'],  
}

In [ ]:
pd.to_datetime('05-10', format='%m-%d', errors='coerce')

In [ ]:
df

In [23]:
# Define the number of top dates you want to find for each year
date_count = 10  # Change this to the desired number of top dates

# Group by 'year', sort by 'amount' within each group, and select top 'date_count' dates
top_dates_by_year = (
    df.groupby('year')
    .apply(lambda x: x.nlargest(date_count, 'amount'))
    .reset_index(drop=True)
)

In [24]:
top_dates_by_year

,payer,country,date,tx,amount,year,payer_country,weekday
0,ELEKTRA (MEXICO),MEXICO,2021-11-27,15885,7.694734e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
1,ELEKTRA (MEXICO),MEXICO,2021-11-20,16708,7.587078e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
2,ELEKTRA (MEXICO),MEXICO,2021-12-18,17487,7.386556e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
3,ELEKTRA (MEXICO),MEXICO,2021-12-04,16378,7.368390e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
4,ELEKTRA (MEXICO),MEXICO,2021-10-09,16635,7.343992e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
5,ELEKTRA (MEXICO),MEXICO,2021-11-26,13915,7.317058e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
6,ELEKTRA (MEXICO),MEXICO,2021-11-21,15996,7.080189e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
7,ELEKTRA (MEXICO),MEXICO,2021-10-30,17589,7.023743e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
8,ELEKTRA (MEXICO),MEXICO,2021-12-11,16249,6.993620e+06,2021,ELEKTRA (MEXICO)_MEXICO,0
9,ELEKTRA (MEXICO),MEXICO,2021-06-19,17047,6.987011e+06,2021,ELEKTRA (MEXICO)_MEXICO,0


In [25]:
df['iso_year'], df['iso_week'], _ = zip(*df['date'].apply(lambda x: x.isocalendar()))

/tmp/ipykernel_20/598475143.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['iso_year'], df['iso_week'], _ = zip(*df['date'].apply(lambda x: x.isocalendar()))


In [26]:
df

,payer,country,date,tx,amount,year,payer_country,weekday,iso_year,iso_week
0,TRANSFERTO - THUNES,BELGIUM,2023-04-03,1,55.55,2023,TRANSFERTO - THUNES_BELGIUM,1,2023,14
2,BANCO GANADERO (BOLIVIA),BOLIVIA,2023-03-27,28,11802.00,2023,BANCO GANADERO (BOLIVIA)_BOLIVIA,1,2023,13
3,BANCO DEL AUSTRO,ECUADOR,2021-09-10,213,78791.19,2021,BANCO DEL AUSTRO_ECUADOR,0,2021,36
4,FEDECACES (EL SALVADOR),EL SALVADOR,2022-04-20,223,148547.00,2022,FEDECACES (EL SALVADOR)_EL SALVADOR,1,2022,16
5,CARIBE EXPRESS,DOMINICAN REPUBLIC,2023-01-23,1117,350050.40,2023,CARIBE EXPRESS_DOMINICAN REPUBLIC,1,2023,4
...,...,...,...,...,...,...,...,...,...,...
101965,BANCO INDUSTRIAL,GUATEMALA,2021-01-02,2005,981210.84,2021,BANCO INDUSTRIAL_GUATEMALA,0,2020,53
101966,BANCOLOMBIA,COLOMBIA,2021-11-16,476,178977.17,2021,BANCOLOMBIA_COLOMBIA,1,2021,46
101967,BANCO DE CREDITO DEL PERU - BCP (UT),PERU,2023-01-23,199,66060.00,2023,BANCO DE CREDITO DEL PERU - BCP (UT)_PERU,1,2023,4
101968,CHEDRAUI (APPRIZA),MEXICO,2022-08-12,23,5437.21,2022,CHEDRAUI (APPRIZA)_MEXICO,0,2022,32


In [27]:
weekly_sales = df.groupby(['iso_year', 'iso_week'])['amount'].sum().reset_index()

In [28]:
weeks_count = 10

top_weeks = (
    weekly_sales
    .groupby('iso_year')
    .apply(lambda x: x.nlargest(weeks_count, 'amount'))
    .reset_index(drop=True)
)

In [29]:
top_weeks

,iso_year,iso_week,amount
0,2020,53,2.735997e+07
1,2021,50,1.648440e+08
2,2021,48,1.521333e+08
3,2021,47,1.495411e+08
4,2021,46,1.488671e+08
5,2021,18,1.458220e+08
6,2021,40,1.450113e+08
7,2021,39,1.448347e+08
8,2021,42,1.432266e+08
9,2021,24,1.429802e+08


In [30]:
# Checking top weeks per year
top_weeks.groupby('iso_year')['iso_week'].agg(lambda x: x.mode())

iso_year
2020                                          53
2021    [18, 24, 39, 40, 42, 43, 46, 47, 48, 50]
2022    [24, 38, 39, 40, 41, 42, 46, 49, 50, 51]
2023    [19, 24, 29, 31, 32, 33, 35, 36, 38, 40]
Name: iso_week, dtype: object

In [32]:
# Semanas 46, 50, 38/39, 24 > Genero lista manual
top_weeks_list = [24, 38, 39, 46, 50]

In [33]:
df['top_week'] = df['iso_week'].isin(top_weeks_list).astype(int)

/tmp/ipykernel_20/2657775018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['top_week'] = df['iso_week'].isin(top_weeks_list).astype(int)


In [34]:
df['top_week'].unique()

array([0, 1])

In [36]:
df[df['top_week'] == 1]

,payer,country,date,tx,amount,year,payer_country,weekday,iso_year,iso_week,top_week
11,CAJA POPULAR MEXICANA (UT),MEXICO,2022-11-17,98,59691.77,2022,CAJA POPULAR MEXICANA (UT)_MEXICO,1,2022,46,1
16,COOPERATIVA FRONTERIZA (UT),HONDURAS,2021-12-16,43,22850.00,2021,COOPERATIVA FRONTERIZA (UT)_HONDURAS,1,2021,50,1
53,WALDOS (APPRIZA),MEXICO,2023-06-12,19,5260.00,2023,WALDOS (APPRIZA)_MEXICO,1,2023,24,1
61,OXXO (TN),MEXICO,2021-09-27,243,17433.91,2021,OXXO (TN)_MEXICO,1,2021,39,1
74,EMPENOS DEL SUR (ANTERIOR BALSAS),MEXICO,2022-06-16,74,27455.13,2022,EMPENOS DEL SUR (ANTERIOR BALSAS)_MEXICO,1,2022,24,1
...,...,...,...,...,...,...,...,...,...,...,...
101907,COOPERATIVA CACIL (UT),HONDURAS,2022-09-22,17,5776.00,2022,COOPERATIVA CACIL (UT)_HONDURAS,1,2022,38,1
101910,PROMERICA,EL SALVADOR,2022-11-17,84,30549.00,2022,PROMERICA_EL SALVADOR,1,2022,46,1
101943,CREDOMATIC,EL SALVADOR,2023-06-12,136,84273.60,2023,CREDOMATIC_EL SALVADOR,1,2023,24,1
101944,DAVIVIENDA,COLOMBIA,2023-06-12,398,113186.38,2023,DAVIVIENDA_COLOMBIA,1,2023,24,1


In [37]:
# Detecting top dates (based on amount)
# Grouping by date and country
sales_summary = df.groupby(['payer_country', 'date'])['amount'].sum().reset_index()
sales_summary = sales_summary.sort_values(by=['country', 'payer_country', 'amount'], ascending=[True, False])

#Top dates by country
dates_selected = 10 # To set
top_dates_by_country = {}
for country in sales_summary['country'].unique():
    top_dates_by_country[country] = sales_summary[sales_summary['country'] == country]['date'][:dates_selected].tolist()

ValueError: Length of ascending (2) != length of by (3)

In [ ]:
top_dates_by_country['MEXICO']

In [ ]:
from collections import Counter

# Listing dates
all_dates_by_country = list(top_dates_by_country.values())
all_dates = [date for sublist in all_dates_by_country for date in sublist]

# Detecting frequency 
date_counter = Counter(all_dates)

# Common dates, for all countries
most_common_dates = date_counter.most_common(dates_selected)

In [38]:
# Setting dates. FORMAT = ['Month-Day','Month-Day']
special_dates_list = [ '07-04', #US independence day 
                      '12-25',
                      '05-11' # Mexico & Colombia - Mother's day
                     '01-01',
                     '01-02',
                     '12-26'] 

In [39]:
special_dates_list

['07-04', '12-25', '05-11']

In [40]:
# Applying format process special_dates function
special_dates_list_processed = [(int(date.split('-')[0]), int(date.split('-')[1])) for date in special_dates_list]

In [41]:
special_dates_list_processed

[(7, 4), (12, 25), (5, 11)]

In [42]:
# To check special dates defined by list
def mark_special_dates(row):
    if (row['date'].month, row['date'].day) in special_dates_list_processed:
        return 1
    else:
        return 0

In [43]:
# Marking special dates in df
df['special_dates'] = df.apply(lambda row: mark_special_dates(row), axis=1)

In [44]:
df[df['special_dates']== 1]

,payer,country,date,tx,amount,year,payer_country,weekday,iso_year,iso_week,top_week,special_dates
193,CEBUANA LHUILLIER,PHILIPPINES,2023-05-11,12,2361.00,2023,CEBUANA LHUILLIER_PHILIPPINES,1,2023,19,0,1
276,MASTERCARD ATM NETWORK,UNITED STATES,2021-12-25,1,140.00,2021,MASTERCARD ATM NETWORK_UNITED STATES,0,2021,51,0,1
323,BANK OF PHILIPPINE ISLANDS (BPI),PHILIPPINES,2023-07-04,20,27831.59,2023,BANK OF PHILIPPINE ISLANDS (BPI)_PHILIPPINES,1,2023,27,0,1
477,BANCO GANADERO (BOLIVIA),BOLIVIA,2023-07-04,4,3350.00,2023,BANCO GANADERO (BOLIVIA)_BOLIVIA,1,2023,27,0,1
586,INTERBANK,PERU,2021-07-04,7,1390.00,2021,INTERBANK_PERU,0,2021,26,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
101523,FARMACIAS GUADALAJARA (UT),MEXICO,2022-05-11,335,55920.45,2022,FARMACIAS GUADALAJARA (UT)_MEXICO,1,2022,19,0,1
101589,MASTERCARD ATM NETWORK,UNITED STATES,2022-05-11,8,1900.00,2022,MASTERCARD ATM NETWORK_UNITED STATES,1,2022,19,0,1
101675,ELEKTRA - MEX (BTS),MEXICO,2021-07-04,8,7963.00,2021,ELEKTRA - MEX (BTS)_MEXICO,0,2021,26,0,1
101766,TELEDOLAR,COSTA RICA,2022-12-25,17,6450.00,2022,TELEDOLAR_COSTA RICA,0,2022,51,0,1


In [45]:
df['special_dates'].unique()

array([0, 1])

In [46]:
# TOP DAYS BY TX
top_count = 10
# Calculate average amount per day per country
avg_amount_per_day = df.groupby(['country', 'date'])['amount'].mean().reset_index()

# Find the top 10 days with the highest average amounts per country
top_days = avg_amount_per_day.groupby('country').apply(lambda x: x.nlargest(top_count, 'amount'))


In [47]:
top_days[top_days['country']=='MEXICO']

country       date        amount
country                                       
MEXICO  18989  MEXICO 2023-09-09  1.537867e+06
        19031  MEXICO 2023-10-21  1.537087e+06
        18982  MEXICO 2023-09-02  1.459775e+06
        19003  MEXICO 2023-09-23  1.446554e+06
        18988  MEXICO 2023-09-08  1.437839e+06
        19017  MEXICO 2023-10-07  1.425614e+06
        18981  MEXICO 2023-09-01  1.385652e+06
        19024  MEXICO 2023-10-14  1.381509e+06
        19004  MEXICO 2023-09-24  1.372996e+06
        19030  MEXICO 2023-10-20  1.358556e+06